In [ ]:
!pip install jieba
import pandas as pd
import jieba

# Load your CSV file
weibo_df = pd.read_csv('weibo_senti_100k_utf8_filtered_four_classes.csv')

# Define a function to segment text
def segment_text(text):
    return ' '.join(jieba.cut(text, cut_all=False))

# Apply the text segmentation function to the 'review_cleaned' column
weibo_df['review_cutted_project'] = weibo_df['review_cleaned'].apply(segment_text)

# Save the DataFrame back to CSV
weibo_df.to_csv('weibo_senti_100k_utf8_filtered_four_classes_processed.csv', index=False)


Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.353 seconds.
DEBUG:jieba:Loading model cost 1.353 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
weibo_df['review_cutted_project'][:10]

0         更博 了 爆照 了 帅 的 呀 就是 越来越 爱 你 生快 傻 缺爱 你 爱 你 爱 你
1      土耳其 的 事要 认真对待 哈哈 否则 直接 开除   很 是 细心 酒店 都 全部 OK 啦
2                                姑娘 都 羡慕 你 呢 还有 招财猫 高兴
3                                                 美爱 你
4                               梦想 有 多 大 舞台 就 有 多 大 鼓掌
5                                         花心 鼓掌   春暖花开
6    某 问答 社区 上 收到 一 大学生 发给 我 的 私信 偶 喜欢 阿姨 偶是 阿姨 控 我...
7    吃货 们 无不 啧啧称奇 好 不 喜欢 PS 写错 一个 字 哈哈                
8    Sweet   MorningFrom   now   onlove   yourselfe...
9    霍思燕 剖腹产 下小江江   老公 落泪 今晨 9 时 霍思燕 产下 一名 男婴 宝宝 重 ...
Name: review_cutted_project, dtype: object

In [ ]:
!pip install tensorflow keras sklearn jieba

import pandas as pd
import jieba
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Dropout, Attention, Lambda
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

  Using cached sklearn-0.0.post11.tar.gz (3.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
# Load the data
df = pd.read_csv('weibo_senti_100k_utf8_filtered_four_classes_processed.csv')

# Segment text with jieba
df['review_cutted_project'] = df['review_cleaned'].apply(lambda x: ' '.join(jieba.cut(x)))

# Preprocessing (assuming the data is already loaded and balanced in df)
X = df['review_cutted_project'].astype(str)
y = df['sentiment_classified_four_classes']
# 1=positive 2=negative 3=Optimistic humorous 4=Pessimistic humorous
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

# Calculate max sequence length
max_seq_length = max([len(x) for x in X_seq])

X_pad = pad_sequences(X_seq, maxlen=max([len(x) for x in X_seq]))

# Encode class values
encoder = LabelEncoder()
encoder.fit(y)
y_enc = encoder.transform(y)
y_dummy = to_categorical(y_enc)

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_dummy, test_size=0.2, random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE
import numpy as np

# Resampling with SMOTE
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Converting the one-hot encoded y_train_resampled back to a 1D array of class labels
y_train_labels = np.argmax(y_train_resampled, axis=1)

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_resampled), y=y_train_resampled)
class_weights_dict = dict(enumerate(class_weights))

In [ ]:
from collections import Counter

class_distribution_before = Counter(y_train)

# Display the class distribution
print("Class distribution after before SMOTE:", class_distribution_before)

# Optionally, you can calculate and display the ratio
total_samples = sum(class_distribution_before.values())
class_ratios = {cls: count / total_samples for cls, count in class_distribution_before.items()}

print("Class ratios after before SMOTE:", class_ratios)

# Count the occurrences of each class in the resampled dataset
class_distribution = Counter(y_train_resampled)

# Display the class distribution
print("Class distribution after applying SMOTE:", class_distribution)

# Optionally, you can calculate and display the ratio
total_samples = sum(class_distribution.values())
class_ratios = {cls: count / total_samples for cls, count in class_distribution.items()}

print("Class ratios after applying SMOTE:", class_ratios)

TypeError: ignored

In [ ]:
# Parameters for the model (adjust these to suit your data)
max_features = 20000  # Adjust to the size of your vocabulary
maxlen = 100          # Adjust to the length that makes sense for your data
embedding_dim = 128
lstm_units = 64

# Model building
input_layer = Input(shape=(maxlen,))
embedding_layer = Embedding(max_features, embedding_dim, input_length=maxlen)(input_layer)
bilstm_layer = Bidirectional(LSTM(lstm_units, return_sequences=True))(embedding_layer)
attention_layer = Attention()([bilstm_layer, bilstm_layer])
lambda_layer = Lambda(lambda x: K.mean(x, axis=1))(attention_layer)
dense_layer = Dense(64, activation='relu')(lambda_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(4, activation='sigmoid')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

In [ ]:
# Model training
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
history = model.fit(X_train, y_train, epochs=1, batch_size=64, validation_split=0.1, callbacks=[early_stopping], verbose=1)

In [ ]:
# Model evaluation
y_pred = model.predict(X_test).argmax(axis=1)
y_true = np.argmax(y_test, axis=1)

# Reverse transform from encoded labels to original labels
y_pred = encoder.inverse_transform(y_pred)
y_true = encoder.inverse_transform(y_true)

# Generate classification report
report = classification_report(y_true, y_pred, target_names=encoder.classes_.astype(str))
print(report)